In [4]:
#!pip install transformers torch pillow pandas matplotlib

import os
import json
from PIL import Image
import torch
import pandas as pd
import matplotlib.pyplot as plt
from transformers import BlipProcessor, BlipForConditionalGeneration


OSError: [WinError 126] The specified module could not be found. Error loading "C:\Users\Pongo\AppData\Roaming\Python\Python311\site-packages\torch\lib\shm.dll" or one of its dependencies.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# BLIP VQA model
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-vqa-base").to(device)

print("✅ Model BLIP VQA loaded on", device)


In [ ]:
with open("vqa_question.json", "r", encoding="utf-8") as f:
    q_data = json.load(f)

questions = q_data["questions"]
print("Daftar pertanyaan VQA:", questions)


In [ ]:
base_dir = "dataset_eval/"
results = []

for exp in os.listdir(base_dir):
    exp_path = os.path.join(base_dir, exp)
    if not os.path.isdir(exp_path):
        continue
    
    for img_file in os.listdir(exp_path):
        if not img_file.lower().endswith((".png", ".jpg", ".jpeg")):
            continue
        
        img_path = os.path.join(exp_path, img_file)
        image = Image.open(img_path).convert("RGB")
        
        for q in questions:
            inputs = processor(image, q, return_tensors="pt").to(device)
            out = model.generate(**inputs)
            answer = processor.decode(out[0], skip_special_tokens=True)
            
            results.append({
                "experiment": exp,
                "image": img_file,
                "question": q,
                "answer": answer
            })

df_vqa = pd.DataFrame(results)
df_vqa.to_csv("vqa_results.csv", index=False)
df_vqa.head()


In [ ]:
# Tampilkan ringkasan jawaban
summary = df_vqa.groupby(["experiment", "question"])["answer"].apply(lambda x: x.value_counts().index[0]).reset_index()

print("Ringkasan jawaban per eksperimen & pertanyaan:")
display(summary)


In [ ]:
# Filter pertanyaan tertentu
q_filter = "Apakah terdapat teks onomatope di dalam gambar?"
subset = df_vqa[df_vqa["question"] == q_filter]

count_answers = subset.groupby(["experiment", "answer"]).size().unstack(fill_value=0)

count_answers.plot(kind="bar", stacked=True, figsize=(10,6))
plt.title("Distribusi Jawaban VQA per Eksperimen")
plt.ylabel("Jumlah Gambar")
plt.xlabel("Eksperimen")
plt.xticks(rotation=45)
plt.legend(title="Jawaban")
plt.show()


In [ ]:
# Ambil contoh 1 gambar per eksperimen dengan jawaban VQA
sample = df_vqa.groupby("experiment").first().reset_index()

fig, axes = plt.subplots(1, len(sample), figsize=(15,5))
for i, row in enumerate(sample.itertuples()):
    img_path = os.path.join(base_dir, row.experiment, row.image)
    img = Image.open(img_path)
    axes[i].imshow(img)
    axes[i].set_title(f"{row.experiment}\n{row.answer[:15]}...")
    axes[i].axis("off")
plt.show()
